In [31]:
!pip install -q tfds-nightly
!pip install nltk
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras import regularizers, initializers, optimizers, callbacks
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
import re
from tqdm import tqdm_notebook
import nltk
nltk.download('stopwords')
from sklearn.model_selection import train_test_split

In [233]:
df = pd.read_csv('train.csv', header = None)
train = df
labels = [2,3]
y = train[labels].values

,0,1,2,3
0,agangsta,A gangsta is usually someone who has money but...,0,1
1,alza,Alma is a word used to describe something craz...,1,0
2,anarchy,Anarchism is possibly the most misunderstood p...,0,1
3,anxiety,A small word which can describe a large set of...,0,1
4,aye,A word that Mexican want to be gangsters use a...,0,1


In [243]:
def clean_text(text, remove_stopwords = True):
    output = ""
    text = str(text).replace("\n", "")
    text = re.sub(r'[^\w\s]','',text).lower()
    text = re.sub('[0-9]', '', text)
    if remove_stopwords:
        text = text.split(" ")
        for word in text:
            if word not in stopwords.words("english"):
                output = output + " " + word
    else:
        output = text
    return str(output.strip())[1:-3].replace("  ", " ")
text_train = train[1]
text_train = list(text_train)
texts = [] 
for line in tqdm_notebook(text_train): 
    texts.append(clean_text(line))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [244]:

tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
data = pad_sequences(sequences, padding = 'post', maxlen = 250)

Vocabulary size: 5234


In [245]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = y[indices]

In [255]:
num_validation_samples = int(0.3*data.shape[0])
x_train = data[: -num_validation_samples].astype('float64')
y_train = labels[: -num_validation_samples].astype('float64')
x_val = data[-num_validation_samples: ].astype('float64')
y_val = labels[-num_validation_samples: ].astype('float64')

In [335]:
embeddings_index = {}
f = open("glove.6B."+str(50)+"d.txt")
for line in f:
    values = line.split()
    word = values[0]
    embeddings_index[word] = np.asarray(values[1:], dtype='float32')
f.close()
embedding_matrix = np.random.random((len(word_index) + 1, 50))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Loading GloVe from: glove.6B.50d.txt ...

In [336]:
sequence_input = Input(shape=(250,), dtype='int32')
embedding_layer = Embedding(len(word_index) + 1,
                           50,
                           weights = [embedding_matrix],
                           input_length = 250,
                           trainable=False,
                           name = 'embeddings')
embedded_sequences = embedding_layer(sequence_input)
embedding_matrix.shape

(11107, 50)

In [337]:
x = LSTM(50, return_sequences=True,name='lstm_layer')(embedded_sequences)
x = GlobalMaxPool1D()(x)
x = Dropout(0.2)(x)
preds = Dense(2, activation="softmax")(x)

In [338]:
model = Model(sequence_input, preds)
model.compile(loss = 'binary_crossentropy',
             optimizer='adam',
             metrics = ['accuracy'])

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 250)]             0         
_________________________________________________________________
embeddings (Embedding)       (None, 250, 50)           555350    
_________________________________________________________________
lstm_layer (LSTM)            (None, 250, 50)           20200     
_________________________________________________________________
global_max_pooling1d (Global (None, 50)                0         
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 2)                 102       
Total params: 575,652
Trainable params: 20,302
Non-trainable params: 555,350
___________________________________________

In [386]:
history = model.fit(x_train, y_train, epochs = 20, batch_size= 8, validation_data=(x_val, y_val))

Epoch 1/20
29/29 [==============================] - 3s 91ms/step - loss: 0.1358 - accuracy: 0.9567 - val_loss: 0.6611 - val_accuracy: 0.7551
Epoch 2/20
29/29 [==============================] - 3s 94ms/step - loss: 0.1346 - accuracy: 0.9610 - val_loss: 0.6953 - val_accuracy: 0.7245
Epoch 3/20
29/29 [==============================] - 3s 91ms/step - loss: 0.0887 - accuracy: 0.9870 - val_loss: 0.8173 - val_accuracy: 0.7551
Epoch 4/20
29/29 [==============================] - 3s 91ms/step - loss: 0.0723 - accuracy: 0.9784 - val_loss: 0.8749 - val_accuracy: 0.7755
Epoch 5/20
29/29 [==============================] - 3s 92ms/step - loss: 0.1549 - accuracy: 0.9697 - val_loss: 0.7371 - val_accuracy: 0.7551
Epoch 6/20
29/29 [==============================] - 3s 93ms/step - loss: 0.1004 - accuracy: 0.9697 - val_loss: 0.8372 - val_accuracy: 0.7347
Epoch 7/20
29/29 [==============================] - 3s 90ms/step - loss: 0.0740 - accuracy: 0.9870 - val_loss: 0.7684 - val_accuracy: 0.7245
Epoch 8/20
29

In [341]:
model.evaluate(x_train, y_train, verbose=0)

[0.1262337863445282, 0.9740259647369385]